In [27]:
import re
import pandas as pd
import numpy as np


In [28]:
class EQUATION:
  vars = []
  vars_VP = []
  vars_VT = []
  names = []
#   def __init__(self, name ,*vars):
#     self.name = name
#     temp =[]
#     for var in vars:
#       temp.append(var)
#     self.vars = temp
#     self.vars_VT = self.vars[0]
#     self.vars_VP = self.vars[1]

  def __init__(self, name, l, r):
    self.name = name
    self.vars_VT = l
    self.vars_VP = r

  #Tổng số chất đang có bên vế trái
  def getNumVar(self): 
    return len(self.vars_VT)

class STEP:
    var_value = -1
    equation = -1
    def __init__(self, var, equation):
      self.var_value = var
      self.equation = equation



In [29]:
class PROBLEM:
  equations = []  # lưu các tri thức (các phương trình)
  knownVar =[] # lưu các hóa chất đã biết
  unknownVar =-1 # hóa chất cần điều chế
  steps = [] # lưu các bước giải bài toán

  def clear(self):
    self.knownVar.clear()
    self.unknownVar.clear()
    self.steps.clear()
    self.unknownVar = -1

  #Cài đặt hóa chất nào đã biết
  def setUnknownVar(self, var):
    self.unknownVar = var
  
  #Cài đặt hóa chất nào chưa biết
#   def setKnownVars(self, *vars):
#     self.knownVars = []
#     for var in vars:
#       self.knownVars.append(var)

  #Cài đặt hóa chất nào chưa biết
  def setKnownVars(self, vars):
    self.knownVars = vars
  
  #Thêm tri thức (phương trình) vào bài toán
#   def addEquation(self, *equations):
#     self.equations = []
#     for equation in equations:
#       self.equations.append(equation)
    #print(self.equations)

  #Thêm tri thức (phương trình) vào bài toán
  def addEquation(self, equations):
    self.equations = equations

  #Số lượng hóa chất đã biết (chỉ tính bên vế trái phương trình)
  def getNumKnownVar(self, equation):
    count = 0
    for var in equation.vars_VT:
      #print(var)
      if (var in self.knownVars):
        count+=1
    return count

  #Trả về yếu tố chưa biết trong phương trình
  def getUnknownVar(self, equation):
    for var in equation.vars_VP:
      if (var not in self.knownVars):
        return var
  
  #Trả về các hóa chát có thể điều chế được, trả về kết quả điều chế
  def getKnownVar(self, equation):
    #print(self.getNumKnownVar(equation))
    if ((self.getNumKnownVar(equation) == equation.getNumVar())):
      return equation.vars_VP
    return -1
  
  #Kích hoạt hóa chất nào có thể điều chế
  def activeVar(self, knownVar):
    for var in knownVar:
      self.knownVars.append(var)
    #print(self.knownVars)

  #Thêm bước giải
  def addStep(self, var, equation):
    self.steps.append((var, equation))

  #Kiểm tra điều chế thành công chưa?
  def isSuccess(self):
    if (self.unknownVar in self.knownVars):
      return True
    return False

  #Giải bài toán
  def solve(self):
    flag = True
    while (flag):
      flag = False
      for equation in self.equations:
        knownVar = self.getKnownVar(equation)
        #print(knownVar)
        if (knownVar != -1):
          self.activeVar(knownVar)
          self.addStep(knownVar, equation)
          flag = True
          if (self.isSuccess()):
            temp = []
            solutions = temp
            for step in self.steps:
              solutions.append(step)
            return solutions
    return ["bài toán không thể giải, hãy bổ sung thêm thông tin hoặc tri thức"]



In [46]:
equations = pd.read_csv(r"C:\Users\Administrator\Desktop\MASTER\HK2\Knowledge Presentation_Thầy Hiển\buoi3\code mau\semantic-mang-ngu-nghia-giai-tam-giac\phuongtrinh.txt",\
                    sep='=',header= None)
equations.columns = ['knownVar','unknownVar']
equations['knownVar'] = equations['knownVar'].apply(lambda x: re.sub('\s+','',x))
equations['unknownVar'] = equations['unknownVar'].apply(lambda x: re.sub('\s+','',x))
#split
equations['knownVar'] = equations.knownVar.apply(lambda x: re.split('\+', x))
equations['unknownVar'] = equations.unknownVar.apply(lambda x: re.split('\+', x)) 
eqList = []
for i in (equations.index.values):
    eq = EQUATION("công thức " + str(i), equations.loc[i][0], equations.loc[i][1])
    #print(eq.names,eq.vars_VP, eq.vars_VT)
    eqList.append(eq) 
    
problem = PROBLEM()
problem.addEquation(eqList)
for i in problem.equations:
    print(i.names,i.vars_VP, i.vars_VT)
    


[] ['NaCl'] ['Na', 'Cl_2']
[] ['FeCl_3'] ['Fe', '3Cl2']
[] ['CuCl2'] ['Cu', 'Cl2']
[] ['HCl', 'HClO'] ['Cl2', 'H2O']
[] ['MnCl2', 'Cl2', '2H2O'] ['MnO2', '4HCl']
[] ['KCl', 'MnCl2', 'H2O', 'Cl2'] ['HCl', 'KMnO4']
[] ['Cl2', 'H2', 'NaOH'] ['NaCl', 'H2O']
[] ['2KCl'] ['K', 'Cl2']
[] ['Na', 'Cl_2'] ['NaCl']
[] ['HCl', 'HClO'] ['Cl_2', 'H_2O']
[] ['SO_2'] ['S', 'O_2']
[] ['SO_3'] ['SO_2', 'O_2']
[] ['H_2SO_4'] ['SO_3', 'H_2O']
[] ['2HCl', 'Na_2SO_4'] ['H_2SO_4', '2NaCl']


In [40]:
  eqList = []
  eqList.append(EQUATION("CT 1", ["Na", "Cl_2"], ["NaCl"]))
  eqList.append(EQUATION("CT 2", ["Fe", "Cl_2"], ["FeCl_3"]))
  eqList.append(EQUATION("CT 3", ["Cu", "Cl_2"], ["CuCl_2"]))
  eqList.append(EQUATION("CT 4", ["Cl_2", "H_2O"], ["HCl", "HClO"]))
  eqList.append(EQUATION("CT 5", ["MnO_2", "HCl"], ["MnCl_2", "Cl_2", "H_2O"]))
  eqList.append(EQUATION("CT 6", ["HCl", "KMnO_4"], ["KCl", "MnCl_2", "H_2O", "Cl_2"]))
  eqList.append(EQUATION("CT 7", ["NaCl", "H_2O"], ["Cl_2", "H_2", "NaOH"]))
  eqList.append(EQUATION("CT 8", ["K", "Cl_2"], ["KCl"]))
  print('eqList', eqList)

eqList [<__main__.EQUATION object at 0x000001D38361A040>, <__main__.EQUATION object at 0x000001D3835D4190>, <__main__.EQUATION object at 0x000001D3835D41F0>, <__main__.EQUATION object at 0x000001D3835D4250>, <__main__.EQUATION object at 0x000001D3835D4040>, <__main__.EQUATION object at 0x000001D3835D42E0>, <__main__.EQUATION object at 0x000001D3835D4280>, <__main__.EQUATION object at 0x000001D3835D4160>]


In [43]:
for i in eqList:
    print(i.names,i.vars_VP, i.vars_VT)

[] ['NaCl'] ['Na', 'Cl_2']
[] ['FeCl_3'] ['Fe', 'Cl_2']
[] ['CuCl_2'] ['Cu', 'Cl_2']
[] ['HCl', 'HClO'] ['Cl_2', 'H_2O']
[] ['MnCl_2', 'Cl_2', 'H_2O'] ['MnO_2', 'HCl']
[] ['KCl', 'MnCl_2', 'H_2O', 'Cl_2'] ['HCl', 'KMnO_4']
[] ['Cl_2', 'H_2', 'NaOH'] ['NaCl', 'H_2O']
[] ['KCl'] ['K', 'Cl_2']


In [32]:
if __name__ == "__main__":
    equations = pd.read_csv(r"C:\Users\Administrator\Desktop\MASTER\HK2\Knowledge Presentation_Thầy Hiển\buoi3\code mau\semantic-mang-ngu-nghia-giai-tam-giac\phuongtrinh.txt",\
                    sep='=',header= None)
    equations.columns = ['knownVar','unknownVar']
    equations['knownVar'] = equations['knownVar'].apply(lambda x: re.sub('\s+','',x))
    equations['unknownVar'] = equations['unknownVar'].apply(lambda x: re.sub('\s+','',x))
    #split
    equations['knownVar'] = equations.knownVar.apply(lambda x: re.split('\+', x))
    equations['unknownVar'] = equations.unknownVar.apply(lambda x: re.split('\+', x)) 
    eqList = []
    for i in (equations.index.values):
        eq = EQUATION("công thức " + str(i), equations.loc[i][0], equations.loc[i][1])
        eqList.append(eq) 
    problem = PROBLEM()
    problem.addEquation(eqList)
    print('eqList', eqList)

# givenChems = input("nguyen lieu dau vao: ").strip().split(",")
    givenChems = ["Na", "Cl_2", "H_2O"]  
    problem.setKnownVars(givenChems)
    #print(givenChems)

  # toBeRefinedChems = input("hoa chat can dieu che: ").strip().split(",")
    toBeRefinedChems = ["NaOH"]
    #print(toBeRefinedChems)

    solutions = []
    for chemistry in toBeRefinedChems:
        problem.setUnknownVar(chemistry)
        print(chemistry)
#         solutions.append(problem.solve())
#         print(solutions)

#     for i,solution in enumerate(solutions):
#         print(f"Ta cần điều chế {toBeRefinedChems[i]}")
#         for index, step in enumerate(solution):
#             print(f"Điều chế lần {index+1}")
#             print(f"Ta điều chế được {step[1].vars_VP} thông qua {step[1].name}")
#             print(f"{step[1].vars_VT} => {step[1].vars_VP}")
#             print("\n")

eqList [<__main__.EQUATION object at 0x000001D3FFC7AB80>, <__main__.EQUATION object at 0x000001D381C87FD0>, <__main__.EQUATION object at 0x000001D381C87EE0>, <__main__.EQUATION object at 0x000001D381C87DF0>, <__main__.EQUATION object at 0x000001D381C87D30>, <__main__.EQUATION object at 0x000001D381C87820>, <__main__.EQUATION object at 0x000001D381C876D0>, <__main__.EQUATION object at 0x000001D381C879D0>, <__main__.EQUATION object at 0x000001D381CED7C0>, <__main__.EQUATION object at 0x000001D381CED7F0>, <__main__.EQUATION object at 0x000001D381CED820>, <__main__.EQUATION object at 0x000001D381CED400>, <__main__.EQUATION object at 0x000001D381CEDF40>, <__main__.EQUATION object at 0x000001D381CEDE20>]
NaOH


In [ ]:
  eqList = []
  eqList.append(EQUATION("CT 1", ["Na", "Cl_2"], ["NaCl"]))
  eqList.append(EQUATION("CT 2", ["Fe", "Cl_2"], ["FeCl_3"]))
  eqList.append(EQUATION("CT 3", ["Cu", "Cl_2"], ["CuCl_2"]))
  eqList.append(EQUATION("CT 4", ["Cl_2", "H_2O"], ["HCl", "HClO"]))
  eqList.append(EQUATION("CT 5", ["MnO_2", "HCl"], ["MnCl_2", "Cl_2", "H_2O"]))
  eqList.append(EQUATION("CT 6", ["HCl", "KMnO_4"], ["KCl", "MnCl_2", "H_2O", "Cl_2"]))
  eqList.append(EQUATION("CT 7", ["NaCl", "H_2O"], ["Cl_2", "H_2", "NaOH"]))
  eqList.append(EQUATION("CT 8", ["K", "Cl_2"], ["KCl"]))
  print('eqList', eqList)

In [19]:
equations

,knownVar,unknownVar
0,"[Na, Cl_2]",[NaCl]
1,"[Fe, 3Cl2]",[FeCl_3]
2,"[Cu, Cl2]",[CuCl2]
3,"[Cl2, H2O]","[HCl, HClO]"
4,"[MnO2, 4HCl]","[MnCl2, Cl2, 2H2O]"
5,"[HCl, KMnO4]","[KCl, MnCl2, H2O, Cl2]"
6,"[NaCl, H2O]","[Cl2, H2, NaOH]"
7,"[K, Cl2]",[2KCl]
8,[NaCl],"[Na, Cl_2]"
9,"[Cl_2, H_2O]","[HCl, HClO]"


In [33]:
eqList = []
for i in (equations.index.values):
    eq = ("công thức " + str(i), equations.loc[i][0], equations.loc[i][1])
    eqList.append(eq) 

In [34]:
eqList

[('công thức 0', ['Na', 'Cl_2'], ['NaCl']),
 ('công thức 1', ['Fe', '3Cl2'], ['FeCl_3']),
 ('công thức 2', ['Cu', 'Cl2'], ['CuCl2']),
 ('công thức 3', ['Cl2', 'H2O'], ['HCl', 'HClO']),
 ('công thức 4', ['MnO2', '4HCl'], ['MnCl2', 'Cl2', '2H2O']),
 ('công thức 5', ['HCl', 'KMnO4'], ['KCl', 'MnCl2', 'H2O', 'Cl2']),
 ('công thức 6', ['NaCl', 'H2O'], ['Cl2', 'H2', 'NaOH']),
 ('công thức 7', ['K', 'Cl2'], ['2KCl']),
 ('công thức 8', ['NaCl'], ['Na', 'Cl_2']),
 ('công thức 9', ['Cl_2', 'H_2O'], ['HCl', 'HClO']),
 ('công thức 10', ['S', 'O_2'], ['SO_2']),
 ('công thức 11', ['SO_2', 'O_2'], ['SO_3']),
 ('công thức 12', ['SO_3', 'H_2O'], ['H_2SO_4']),
 ('công thức 13', ['H_2SO_4', '2NaCl'], ['2HCl', 'Na_2SO_4'])]

In [38]:
  eqList = []
  eqList.append(["CT 1", ["Na", "Cl_2"], ["NaCl"]])
  eqList.append("CT 2", ["Fe", "Cl_2"], ["FeCl_3"])
  eqList.append("CT 3", ["Cu", "Cl_2"], ["CuCl_2"])
  eqList.append("CT 4", ["Cl_2", "H_2O"], ["HCl", "HClO"])
  eqList.append("CT 5", ["MnO_2", "HCl"], ["MnCl_2", "Cl_2", "H_2O"])
  eqList.append("CT 6", ["HCl", "KMnO_4"], ["KCl", "MnCl_2", "H_2O", "Cl_2"])
  eqList.append("CT 7", ["NaCl", "H_2O"], ["Cl_2", "H_2", "NaOH"])
  eqList.append("CT 8", ["K", "Cl_2"], ["KCl"])
  print( eqList)

TypeError: append() takes exactly one argument (3 given)